In [1]:
import psycopg2 as pg

from io import StringIO
import pandas as pd
import os
from datetime import datetime
from datetime import date
import numpy as np
from tqdm.notebook import tqdm
import pyspark
import calendar
from pyspark.sql import SparkSession
from pyspark.sql import Row
from datetime import timedelta
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import array_contains
from pyspark.sql.functions import date_format
import warnings
warnings.filterwarnings('ignore')
import time

## FINAL CODE

In [6]:
################################# CONVERTING 1-min TO DIFFERENT TIMEFRAMES - UNDERLYING ##################################
def EOD_underlying(ddf,symbol):
    print("CONVERTING TO EOD")
    final_df = ddf.copy()
    final_df['Date'] = pd.to_datetime(final_df['Date'], format='mixed',dayfirst=True)
    final_df = final_df[(final_df['Time']>=time1) & (final_df['Time']<=time2)]
    final_df.reset_index(drop=True,inplace=True)
    final_df = final_df.sort_values(by=['Date'])
    final_df = final_df.rename(columns={'Time' : 'Timestamp',
                                        'EQ_Open' : 'Adj_Open',
                                        'EQ_High' : 'Adj_High',
                                        'EQ_Low' : 'Adj_Low',
                                        'EQ_Close' : 'Adj_Close',
                                        'EQ_Volume' : 'Adj_Volume'})
    final_df['Date'] = pd.to_datetime(final_df['Date'],format='mixed',dayfirst=True).dt.date
    final_df = final_df.sort_values(by=['Date', 'Timestamp'])
    final_df['Date'] = final_df['Date'].astype(str)
    final_df['Timestamp'] = final_df['Timestamp'].astype(str)
    final_df['Datetime'] = pd.to_datetime(final_df['Date'] + ' ' + final_df['Timestamp'], format='mixed',dayfirst=True)

    final_df = final_df.set_index("Datetime")

    ddf = final_df.groupby(['Date', pd.Grouper(freq='B')]).agg({"Adj_Open" : "first", 
                                                          "Adj_High" : "max",
                                                          "Adj_Low" : "min",
                                                          "Adj_Close" : "last", 
                                                          'Adj_Volume' : 'sum'})
    ddf.columns = ["Adj_Open", "Adj_High", "Adj_Low", "Adj_Close", 'Adj_Volume']
    ddf = ddf.reset_index()
    ddf['Ticker'] = f"{symbol}".upper()+'.EQ-NSE'

    ddf = ddf.sort_values(by=['Datetime'])
    ddf.reset_index(drop=True,inplace=True)
    ddf = ddf.rename(columns={'Adj_Open':'EQ_Open','Adj_High':'EQ_High','Adj_Low':'EQ_Low','Adj_Close':'EQ_Close','Adj_Volume':'EQ_Volume'})
    ddf = ddf[['Ticker','Date','EQ_Open','EQ_High','EQ_Low','EQ_Close','EQ_Volume']]
    ddf.to_csv(fr"C:\\users\\{admin_path}\\desktop\\Stocks_Data\\{symbol.upper()}_EQ_EOD.csv",index=False)

def fifteen_underlying(ddf,symbol):
    print("CONVERTING TO 15min")
    final_df = ddf.copy()
    final_df['Date'] = pd.to_datetime(final_df['Date'],format='mixed', dayfirst=True)
    final_df = final_df[(final_df['Time']>=time1) & (final_df['Time']<=time2)]
    final_df.reset_index(drop=True,inplace=True)
    final_df = final_df.sort_values(by=['Date'])
    final_df = final_df.rename(columns={'Time' : 'Timestamp',
                                        'EQ_Open' : 'Adj_Open',
                                        'EQ_High' : 'Adj_High',
                                        'EQ_Low' : 'Adj_Low',
                                        'EQ_Close' : 'Adj_Close',
                                        'EQ_Volume' : 'Adj_Volume'})
    final_df['Date'] = pd.to_datetime(final_df['Date'],format='mixed',dayfirst=True).dt.date
    final_df = final_df.sort_values(by=['Date', 'Timestamp'])
    final_df['Date'] = final_df['Date'].astype(str)
    final_df['Timestamp'] = final_df['Timestamp'].astype(str)
    final_df['Datetime'] = pd.to_datetime(final_df['Date'] + ' ' + final_df['Timestamp'],format='mixed', dayfirst=True)

    final_df = final_df.set_index("Datetime")

    ddf = final_df.groupby(['Date', pd.Grouper(freq='15min')]).agg({"Adj_Open" : "first", 
                                                          "Adj_High" : "max",
                                                          "Adj_Low" : "min",
                                                          "Adj_Close" : "last", 
                                                          'Adj_Volume' : 'sum'})
    ddf.columns = ["Adj_Open", "Adj_High", "Adj_Low", "Adj_Close", 'Adj_Volume']
    ddf = ddf.reset_index()
    ddf['Ticker'] = f"{symbol}".upper()+'.EQ-NSE'

    ddf = ddf.sort_values(by=['Datetime'])
    ddf['Time'] = pd.to_datetime(ddf['Datetime']).dt.time
    ddf.reset_index(drop=True,inplace=True)
    ddf = ddf.rename(columns={'Adj_Open':'EQ_Open','Adj_High':'EQ_High','Adj_Low':'EQ_Low','Adj_Close':'EQ_Close','Adj_Volume':'EQ_Volume'})
    ddf = ddf[['Ticker','Date','Time','EQ_Open','EQ_High','EQ_Low','EQ_Close','EQ_Volume']]
    ddf.to_csv(fr"C:\\users\\{admin_path}\\desktop\\Stocks_Data\\{symbol.upper()}_EQ_15min.csv",index=False)

def five_underlying(ddf,symbol):
    print("CONVERTING TO 5min")
    final_df = ddf.copy()
    final_df['Date'] = pd.to_datetime(final_df['Date'], format='mixed',dayfirst=True)
    final_df = final_df[(final_df['Time']>=time1) & (final_df['Time']<=time2)]
    final_df.reset_index(drop=True,inplace=True)
    final_df = final_df.sort_values(by=['Date'])
    final_df = final_df.rename(columns={'Time' : 'Timestamp',
                                        'EQ_Open' : 'Adj_Open',
                                        'EQ_High' : 'Adj_High',
                                        'EQ_Low' : 'Adj_Low',
                                        'EQ_Close' : 'Adj_Close',
                                        'EQ_Volume' : 'Adj_Volume'})
    final_df['Date'] = pd.to_datetime(final_df['Date'],format='mixed',dayfirst=True).dt.date
    final_df = final_df.sort_values(by=['Date', 'Timestamp'])
    final_df['Date'] = final_df['Date'].astype(str)
    final_df['Timestamp'] = final_df['Timestamp'].astype(str)
    final_df['Datetime'] = pd.to_datetime(final_df['Date'] + ' ' + final_df['Timestamp'], format='mixed',dayfirst=True)

    final_df = final_df.set_index("Datetime")

    ddf = final_df.groupby(['Date', pd.Grouper(freq='5min')]).agg({"Adj_Open" : "first", 
                                                          "Adj_High" : "max",
                                                          "Adj_Low" : "min",
                                                          "Adj_Close" : "last", 
                                                          'Adj_Volume' : 'sum'})
    ddf.columns = ["Adj_Open", "Adj_High", "Adj_Low", "Adj_Close", 'Adj_Volume']
    ddf = ddf.reset_index()
    ddf['Ticker'] = f"{symbol}".upper()+'.EQ-NSE'

    ddf = ddf.sort_values(by=['Datetime'])
    ddf['Time'] = pd.to_datetime(ddf['Datetime']).dt.time
    ddf.reset_index(drop=True,inplace=True)
    ddf = ddf.rename(columns={'Adj_Open':'EQ_Open','Adj_High':'EQ_High','Adj_Low':'EQ_Low','Adj_Close':'EQ_Close','Adj_Volume':'EQ_Volume'})
    ddf = ddf[['Ticker','Date','Time','EQ_Open','EQ_High','EQ_Low','EQ_Close','EQ_Volume']]
    ddf.to_csv(fr"C:\\users\\{admin_path}\\desktop\\Stocks_Data\\{symbol.upper()}_EQ_5min.csv",index=False)

def one_underlying(ddf,symbol):
    final_df = ddf.copy()
    final_df['Date'] = pd.to_datetime(final_df['Date'],format='mixed', dayfirst=True)
    final_df = final_df[(final_df['Time']>=time1) & (final_df['Time']<=time2)]
    final_df = final_df.sort_values(by=['Date','Time'])
    final_df.reset_index(drop=True,inplace=True)
    final_df = final_df[['Ticker','Date','Time','EQ_Open','EQ_High','EQ_Low','EQ_Close','EQ_Volume']]
    final_df.to_csv(fr"C:\\users\\{admin_path}\\desktop\\Stocks_Data\\{symbol.upper()}_EQ_1min.csv",index=False)

################################# CONVERTING 1-min TO DIFFERENT TIMEFRAMES - OPTIONS DATA ##################################
def EOD(ddf,symbol,schema):
    print("CONVERTING TO EOD")
    ddf['Date'] = pd.to_datetime(ddf['Date'], format='mixed',dayfirst=True)
    ddf = ddf.sort_values(by=['Date'])
    
    j='-' + schema
    
    final_df = ddf.copy()
    final_df['Final_strike'] = final_df['Ticker'].str.replace(j, '')
    final_df['Final_strike'] = final_df['Final_strike'].str.replace(f'{symbol.upper()}','').str.replace('CE', '').str.replace('PE', '')
    final_df['Final_strike'] = final_df['Final_strike'].astype(float)
    final_df['Option_Type'] = final_df['Ticker'].str[-2:]
    
    final_df = final_df.rename(columns={'Time' : 'Timestamp',
                                        'Open' : 'Adj_Open',
                                        'High' : 'Adj_High',
                                        'Low' : 'Adj_Low',
                                        'Close' : 'Adj_Close',
                                        'Volume' : 'Adj_Volume',
                                        'Open_Interest' : 'Adj_OI',        
                                        'Option_type' : 'Option_Type'})
    final_df['Date'] = pd.to_datetime(final_df['Date'],format='mixed',dayfirst=True).dt.date
    final_df = final_df.sort_values(by=['Date', 'Timestamp'])
    final_df['Date'] = final_df['Date'].astype(str)
    final_df['Timestamp'] = final_df['Timestamp'].astype(str)
    final_df['Datetime'] = pd.to_datetime(final_df['Date'] + ' ' + final_df['Timestamp'], format='mixed',dayfirst=True)

    final_df = final_df.set_index("Datetime")
    final_df['Adj_OI_1'] = final_df['Adj_OI']

    df_eod = final_df.groupby(['Final_strike', 'Option_Type', pd.Grouper(freq='B')]).agg({"Adj_Open" : "first", 
                                                          "Adj_High" : "max",
                                                          "Adj_Low" : "min",
                                                          "Adj_Close" : "last", 
                                                          'Adj_Volume' : 'sum',
                                                          'Adj_OI' : 'first',
                                                          'Adj_OI_1' : 'last'})
    df_eod.columns = ["Adj_Open", "Adj_High", "Adj_Low", "Adj_Close", 'Adj_Volume', 'First_OI', 'Last_OI']
    df_eod = df_eod.reset_index()
    df_eod['rem'] = df_eod['Final_strike']%df_eod['Final_strike'].astype(int)
    df_eod.loc[df_eod['rem'] == 0, 'Ticker'] = symbol.upper() + j +  df_eod['Final_strike'].astype(int).astype(str) + df_eod['Option_Type']
    df_eod.loc[df_eod['rem'] != 0, 'Ticker'] = symbol.upper() + j + df_eod['Final_strike'].round(2).astype(str) + df_eod['Option_Type'] 

    df_eod = df_eod.sort_values(by=['Datetime', 'Final_strike'])
    df_eod = df_eod.rename(columns={'Datetime' : 'Date'})
    df_eod['Date'] = pd.to_datetime(df_eod['Date'],format='mixed',dayfirst=True)
    ## CHECKING IF NULL VALUES
    df_eod['New_OI'] = df_eod['Last_OI']
    df_eod = df_eod.rename(columns={'Adj_Open':'Open','Adj_High':'High','Adj_Low':'Low','Adj_Close':'Close','Adj_Volume':'Volume','New_OI':'Open_Interest'})
    df_eod = df_eod.drop(['First_OI','Last_OI','Option_Type','Final_strike','rem'],axis=1)
    df_eod = df_eod[['Ticker','Date','Open','High','Low','Close','Volume','Open_Interest']]
    df_eod.reset_index(drop=True,inplace=True)
    df_eod.to_csv(fr"C:\\users\\{admin_path}\\desktop\\Stocks_Data\\{symbol.upper()}_Opt_EOD.csv", mode='a', header = not os.path.exists(fr"C:\\users\\{admin_path}\\desktop\\Stocks_Data\\{symbol.upper()}_Opt_EOD.csv"), index=False)
    
def fifteen(ddf,symbol,schema):
    print("CONVERTING TO 15Min")
    ddf['Date'] = pd.to_datetime(ddf['Date'], format='mixed',dayfirst=True)
    ddf = ddf.sort_values(by=['Date'])
    
    j='-' + schema
    
    final_df = ddf.copy()
    final_df['Final_strike'] = final_df['Ticker'].str.replace(j, '')
    final_df['Final_strike'] = final_df['Final_strike'].str.replace(f'{symbol.upper()}','').str.replace('CE', '').str.replace('PE', '')
    final_df['Final_strike'] = final_df['Final_strike'].astype(float)
    final_df['Option_Type'] = final_df['Ticker'].str[-2:]
    
    final_df = final_df.rename(columns={'Time' : 'Timestamp',
                                        'Open' : 'Adj_Open',
                                        'High' : 'Adj_High',
                                        'Low' : 'Adj_Low',
                                        'Close' : 'Adj_Close',
                                        'Volume' : 'Adj_Volume',
                                        'Open_Interest' : 'Adj_OI',        
                                        'Option_type' : 'Option_Type'})
    final_df['Date'] = pd.to_datetime(final_df['Date'],format='mixed',dayfirst=True).dt.date
    final_df = final_df.sort_values(by=['Date', 'Timestamp'])
    final_df['Date'] = final_df['Date'].astype(str)
    final_df['Timestamp'] = final_df['Timestamp'].astype(str)
    final_df['Datetime'] = pd.to_datetime(final_df['Date'] + ' ' + final_df['Timestamp'], format='mixed',dayfirst=True)

    final_df = final_df.set_index("Datetime")
    final_df['Adj_OI_1'] = final_df['Adj_OI']

    df_eod = final_df.groupby(['Final_strike', 'Option_Type', pd.Grouper(freq='15min')]).agg({"Adj_Open" : "first", 
                                                          "Adj_High" : "max",
                                                          "Adj_Low" : "min",
                                                          "Adj_Close" : "last", 
                                                          'Adj_Volume' : 'sum',
                                                          'Adj_OI' : 'first',
                                                          'Adj_OI_1' : 'last'})
    df_eod.columns = ["Adj_Open", "Adj_High", "Adj_Low", "Adj_Close", 'Adj_Volume', 'First_OI', 'Last_OI']
    df_eod = df_eod.reset_index()
    df_eod['rem'] = df_eod['Final_strike']%df_eod['Final_strike'].astype(int)
    df_eod.loc[df_eod['rem'] == 0, 'Ticker'] = symbol.upper() + j +  df_eod['Final_strike'].astype(int).astype(str) + df_eod['Option_Type']
    df_eod.loc[df_eod['rem'] != 0, 'Ticker'] = symbol.upper() + j + df_eod['Final_strike'].round(2).astype(str) + df_eod['Option_Type'] 

    df_eod = df_eod.sort_values(by=['Datetime', 'Final_strike'])
    df_eod = df_eod.rename(columns={'Datetime' : 'Date'})
    df_eod['Date'] = pd.to_datetime(df_eod['Date'],format='mixed',dayfirst=True)
    ## CHECKING IF NULL VALUES
    df_eod['New_OI'] = df_eod['Last_OI']
    df_eod = df_eod.rename(columns={'Adj_Open':'Open','Adj_High':'High','Adj_Low':'Low','Adj_Close':'Close','Adj_Volume':'Volume','New_OI':'Open_Interest'})
    df_eod = df_eod.drop(['First_OI','Last_OI','Option_Type','Final_strike','rem'],axis=1)
    df_eod = df_eod[['Ticker','Date','Open','High','Low','Close','Volume','Open_Interest']]
    df_eod.reset_index(drop=True,inplace=True)
    df_eod.to_csv(fr"C:\\users\\{admin_path}\\desktop\\Stocks_Data\\{symbol.upper()}_Opt_15min.csv", mode='a', header = not os.path.exists(fr"C:\\users\\{admin_path}\\desktop\\Stocks_Data\\{symbol.upper()}_Opt_15min.csv"), index=False)

def five(ddf,symbol,schema):
    print("CONVERTING TO 5Min")
    ddf['Date'] = pd.to_datetime(ddf['Date'], format='mixed',dayfirst=True)
    ddf = ddf.sort_values(by=['Date'])
    
    j='-' + schema
    
    final_df = ddf.copy()
    final_df['Final_strike'] = final_df['Ticker'].str.replace(j, '')
    final_df['Final_strike'] = final_df['Final_strike'].str.replace(f'{symbol.upper()}','').str.replace('CE', '').str.replace('PE', '')
    final_df['Final_strike'] = final_df['Final_strike'].astype(float)
    final_df['Option_Type'] = final_df['Ticker'].str[-2:]
    
    final_df = final_df.rename(columns={'Time' : 'Timestamp',
                                        'Open' : 'Adj_Open',
                                        'High' : 'Adj_High',
                                        'Low' : 'Adj_Low',
                                        'Close' : 'Adj_Close',
                                        'Volume' : 'Adj_Volume',
                                        'Open_Interest' : 'Adj_OI',        
                                        'Option_type' : 'Option_Type'})
    final_df['Date'] = pd.to_datetime(final_df['Date'],format='mixed',dayfirst=True).dt.date
    final_df = final_df.sort_values(by=['Date', 'Timestamp'])
    final_df['Date'] = final_df['Date'].astype(str)
    final_df['Timestamp'] = final_df['Timestamp'].astype(str)
    final_df['Datetime'] = pd.to_datetime(final_df['Date'] + ' ' + final_df['Timestamp'], format='mixed',dayfirst=True)

    final_df = final_df.set_index("Datetime")
    final_df['Adj_OI_1'] = final_df['Adj_OI']

    df_eod = final_df.groupby(['Final_strike', 'Option_Type', pd.Grouper(freq='5min')]).agg({"Adj_Open" : "first", 
                                                          "Adj_High" : "max",
                                                          "Adj_Low" : "min",
                                                          "Adj_Close" : "last", 
                                                          'Adj_Volume' : 'sum',
                                                          'Adj_OI' : 'first',
                                                          'Adj_OI_1' : 'last'})
    df_eod.columns = ["Adj_Open", "Adj_High", "Adj_Low", "Adj_Close", 'Adj_Volume', 'First_OI', 'Last_OI']
    df_eod = df_eod.reset_index()
    df_eod['rem'] = df_eod['Final_strike']%df_eod['Final_strike'].astype(int)
    df_eod.loc[df_eod['rem'] == 0, 'Ticker'] = symbol.upper() + j +  df_eod['Final_strike'].astype(int).astype(str) + df_eod['Option_Type']
    df_eod.loc[df_eod['rem'] != 0, 'Ticker'] = symbol.upper() + j + df_eod['Final_strike'].round(2).astype(str) + df_eod['Option_Type'] 

    df_eod = df_eod.sort_values(by=['Datetime', 'Final_strike'])
    df_eod = df_eod.rename(columns={'Datetime' : 'Date'})
    df_eod['Date'] = pd.to_datetime(df_eod['Date'],format='mixed',dayfirst=True)
    ## CHECKING IF NULL VALUES
    df_eod['New_OI'] = df_eod['Last_OI']
    df_eod = df_eod.rename(columns={'Adj_Open':'Open','Adj_High':'High','Adj_Low':'Low','Adj_Close':'Close','Adj_Volume':'Volume','New_OI':'Open_Interest'})
    df_eod = df_eod.drop(['First_OI','Last_OI','Option_Type','Final_strike','rem'],axis=1)
    df_eod = df_eod[['Ticker','Date','Open','High','Low','Close','Volume','Open_Interest']]
    df_eod.reset_index(drop=True,inplace=True)
    df_eod.to_csv(fr"C:\\users\\{admin_path}\\desktop\\Stocks_Data\\{symbol.upper()}_Opt_5min.csv", mode='a', header = not os.path.exists(fr"C:\\users\\{admin_path}\\desktop\\Stocks_Data\\{symbol.upper()}_Opt_5min.csv"), index=False)

def one(ddf,symbol,schema):
    
    ddf['Date'] = pd.to_datetime(ddf['Date'], format='mixed',dayfirst=True)
    ddf = ddf.sort_values(by=['Date'])
    ddf = ddf[(ddf['Time']>=time1) & ((ddf['Time']<=time2))]
    ddf = ddf.sort_values(by=['Date','Time'])
    ddf.reset_index(drop=True,inplace=True)
    ddf = ddf[['Ticker','Date','Time','Open','High','Low','Close','Volume','Open_Interest']]
    ddf.to_csv(fr"C:\\users\\{admin_path}\\desktop\\Stocks_Data\\{symbol.upper()}_Opt_1min.csv", mode='a', header = not os.path.exists(fr"C:\\users\\{admin_path}\\desktop\\Stocks_Data\\{symbol.upper()}_Opt_1min.csv"), index=False)

####################################### UNDERLYING DATA FUNCTION ##########################################
def underlying_data(symbol,date1,date2,conversion):
    print("\nFetching Stock Equity Data")
    ddf = pd.DataFrame()
    engine = pg.connect("dbname='StockEQ' user='postgres' host='swandatabase.cfehmk2wtejq.ap-south-1.rds.amazonaws.com' port='5432' password='swancap123'")
    st_t = time.time()
    ddf = pd.read_sql(f'select * from "AdjustedData"."{symbol.upper()}" where "Date" between \'{date1}\' and \'{date2}\'', con=engine)
    ddf['Ticker'] = f'{symbol.upper()}' + '.EQ-NSE'
    ddf = ddf.sort_values(by=["Date",'Time'])
    
    if conversion == 'E':
        EOD_underlying(ddf,symbol)
    elif conversion == '15':
        fifteen_underlying(ddf,symbol)
    elif conversion == '5':
        five_underlying(ddf,symbol)
    elif conversion == '1':
        one_underlying(ddf,symbol)
        
    et_t = time.time()
    print("Time taken to process",et_t-st_t)
    engine.close()

####################################### OPTION DATA FUNCTION ##########################################        
def option_data(symbol,date1,date2,conversion,schema):
    print("\nFetching Stock Option Data")
    date1 = datetime.strptime(date1, "%Y-%m-%d").date()
    date2 = datetime.strptime(date2, "%Y-%m-%d").date()
    year1 = date1.year
    year2 = date2.year
    
    st_t = time.time()
    for i in tqdm(range(int(year1),int(year2)+1)):
        ddate1 = '-01-01'
        ddate2 = '-12-31'
        year_start = str(str(i)+ddate1)
        year_end = str(str(i)+ddate2)
        year_start = datetime.strptime(year_start, "%Y-%m-%d").date()
        year_end = datetime.strptime(year_end, "%Y-%m-%d").date()
        if date1 > year_start :
            year_start = date1
        else:
            year_start = year_start
        if year_end > date2 :
            year_end = date2
        else:
            year_end = year_end
        
        ddf = pd.DataFrame()
        
        try:    
            engine = pg.connect("dbname='StockOptions' user='postgres' host='swandatabase.cfehmk2wtejq.ap-south-1.rds.amazonaws.com' port='5432' password='swancap123'")
            print("Generating data from ", year_start , " to " , year_end)
            ddf = pd.read_sql(f'select * from "Monthly{schema}"."{symbol.upper()}-{schema}" where "Date" between \'{year_start}\' and \'{year_end}\'', con=engine)
            ddf = ddf.sort_values(by=["Date",'Time'])
            ddf.reset_index(drop=True,inplace=True)
    #         ddf.to_csv(rf"C:\\users\\admin\\desktop\\Stocks_Data\\{symbol.upper()}_"+str(i)+".csv",index=False)
            if conversion == 'E':
                EOD(ddf,symbol,schema)
            elif conversion == '15':
                fifteen(ddf,symbol,schema)
            elif conversion == '5':
                five(ddf,symbol,schema)
            elif conversion == '1':
                one(ddf,symbol,schema)
        except:
            print(f'"Monthly{schema}"."{symbol.upper()}-{schema}" does not exist')
            
    et_t = time.time()
    print("Time taken to process",et_t-st_t)
    engine.close()
    
###################################### MAIN CODE STARTS FROM HERE ############################################
admin_path = 'admin'

number_of_symbols = int(input("Enter number of symbols "))
symbol_list = []

for i in range(number_of_symbols):
    symbol_input = input('Enter the Symbol ')
    symbol_list.append(symbol_input)
    
data = input("Enter O for Options data, U for Underlying data, B for Both the data, E for Exiting ")

time1 = datetime.strptime('09:15:00','%H:%M:%S').time()
time2 = datetime.strptime('15:30:00','%H:%M:%S').time()

## CREATING A DIRECTORY OF THE REQUIRED INDEX AND SCHEMA
if not os.path.exists(fr"C:\\users\\{admin_path}\\desktop\\Stocks_Data\\"):
    os.makedirs(fr"C:\users\\{admin_path}\desktop\\Stocks_Data\\")

#################################### TAKING INPUTS FOR SYMBOLS, DATE RANGE AND TIMEFRAME #############################

if data == 'o' or data == 'O' or data == 'b' or data == 'B':
#     symbol = input('Enter the Symbol ')
    schema = input("Enter schema (I/II/III) - ")
    date1 = input("Enter Start date as YYYY-MM-DD ")
    date2 = input("Enter End date as YYYY-MM-DD ")
    conversion = input("Enter 1 for 1 minute, 5 for 5 minutes, 15 for 15 minutes, E for EOD ")
    
elif data == 'u' or data == 'U':
    date1 = input("Enter Start date as YYYY-MM-DD ")
    date2 = input("Enter End date as YYYY-MM-DD ")
    conversion = input("Enter 1 for 1 minute, 5 for 5 minutes, 15 for 15 minutes, E for EOD ")
    
elif data == 'e' or data == 'E':
    print("Exit!")

else:
    print("Wrong option")

st=time.time()

for i in range(number_of_symbols):
    
    if data == 'o' or data == 'O':
        option_data(symbol_list[i],date1,date2,conversion,schema)

    elif data == 'U' or data == 'u':
        underlying_data(symbol_list[i],date1,date2,conversion)

    elif data == 'b' or data == 'B':
        option_data(symbol_list[i],date1,date2,conversion,schema)
        underlying_data(symbol_list[i],date1,date2,conversion)
    
et = time.time()
print("\nTotal time taken ",et-st)

Enter number of symbols  1
Enter the Symbol  reliance
Enter O for Options data, U for Underlying data, B for Both the data, E for Exiting  u
Enter Start date as YYYY-MM-DD  2023-01-01
Enter End date as YYYY-MM-DD  2023-01-03
Enter 1 for 1 minute, 5 for 5 minutes, 15 for 15 minutes, E for EOD  1



Fetching Stock Equity Data
Time taken to process 0.3672447204589844

Total time taken  0.4443073272705078


In [31]:
# conn = psycopg2.connect(host='swandatabase.cfehmk2wtejq.ap-south-1.rds.amazonaws.com', dbname='StockEQ',
#                         user='postgres', password='swancap123')
# cursor = conn.cursor()

# cursor.execute("""SELECT table_name  FROM information_schema.tables WHERE TABLE_TYPE='BASE TABLE' AND TABLE_SCHEMA = 'AdjustedData'""") # "rel" is short for relation.
# symbols = [i[0] for i in cursor.fetchall()]
# conn.close()


# symbols

In [21]:
# len(symbols)

## CODE TO FETCH STOCK OPTIONS + UNDERLYING

In [20]:
################################# CONVERTING 1-min TO DIFFERENT TIMEFRAMES - UNDERLYING ##################################
def EOD_underlying(ddf,symbol):
    print("CONVERTING TO EOD")
    final_df = ddf.copy()
    final_df['Date'] = pd.to_datetime(final_df['Date'], dayfirst=True)
    final_df = final_df[(final_df['Time']>=time1) & (final_df['Time']<=time2)]
    final_df.reset_index(drop=True,inplace=True)
    final_df = final_df.sort_values(by=['Date'])
    final_df = final_df.rename(columns={'Time' : 'Timestamp',
                                        'EQ_Open' : 'Adj_Open',
                                        'EQ_High' : 'Adj_High',
                                        'EQ_Low' : 'Adj_Low',
                                        'EQ_Close' : 'Adj_Close',
                                        'EQ_Volume' : 'Adj_Volume'})
    final_df['Date'] = pd.to_datetime(final_df['Date'],dayfirst=True).dt.date
    final_df = final_df.sort_values(by=['Date', 'Timestamp'])
    final_df['Date'] = final_df['Date'].astype(str)
    final_df['Timestamp'] = final_df['Timestamp'].astype(str)
    final_df['Datetime'] = pd.to_datetime(final_df['Date'] + ' ' + final_df['Timestamp'], dayfirst=True)

    final_df = final_df.set_index("Datetime")

    ddf = final_df.groupby(['Date', pd.Grouper(freq='B')]).agg({"Adj_Open" : "first", 
                                                          "Adj_High" : "max",
                                                          "Adj_Low" : "min",
                                                          "Adj_Close" : "last", 
                                                          'Adj_Volume' : 'sum'})
    ddf.columns = ["Adj_Open", "Adj_High", "Adj_Low", "Adj_Close", 'Adj_Volume']
    ddf = ddf.reset_index()
    ddf['Ticker'] = f"{symbol}".upper()+'.EQ-NSE'

    ddf = ddf.sort_values(by=['Datetime'])
    ddf.reset_index(drop=True,inplace=True)
    ddf = ddf.rename(columns={'Adj_Open':'EQ_Open','Adj_High':'EQ_High','Adj_Low':'EQ_Low','Adj_Close':'EQ_Close','Adj_Volume':'EQ_Volume'})
    ddf = ddf[['Ticker','Date','EQ_Open','EQ_High','EQ_Low','EQ_Close','EQ_Volume']]
    ddf.to_csv(fr"C:\\users\\{admin_path}\\desktop\\Stocks_Data\\{symbol.upper()}_EQ_EOD.csv",index=False)

def fifteen_underlying(ddf,symbol):
    print("CONVERTING TO 15min")
    final_df = ddf.copy()
    final_df['Date'] = pd.to_datetime(final_df['Date'], dayfirst=True)
    final_df = final_df[(final_df['Time']>=time1) & (final_df['Time']<=time2)]
    final_df.reset_index(drop=True,inplace=True)
    final_df = final_df.sort_values(by=['Date'])
    final_df = final_df.rename(columns={'Time' : 'Timestamp',
                                        'EQ_Open' : 'Adj_Open',
                                        'EQ_High' : 'Adj_High',
                                        'EQ_Low' : 'Adj_Low',
                                        'EQ_Close' : 'Adj_Close',
                                        'EQ_Volume' : 'Adj_Volume'})
    final_df['Date'] = pd.to_datetime(final_df['Date'],dayfirst=True).dt.date
    final_df = final_df.sort_values(by=['Date', 'Timestamp'])
    final_df['Date'] = final_df['Date'].astype(str)
    final_df['Timestamp'] = final_df['Timestamp'].astype(str)
    final_df['Datetime'] = pd.to_datetime(final_df['Date'] + ' ' + final_df['Timestamp'], dayfirst=True)

    final_df = final_df.set_index("Datetime")

    ddf = final_df.groupby(['Date', pd.Grouper(freq='15min')]).agg({"Adj_Open" : "first", 
                                                          "Adj_High" : "max",
                                                          "Adj_Low" : "min",
                                                          "Adj_Close" : "last", 
                                                          'Adj_Volume' : 'sum'})
    ddf.columns = ["Adj_Open", "Adj_High", "Adj_Low", "Adj_Close", 'Adj_Volume']
    ddf = ddf.reset_index()
    ddf['Ticker'] = f"{symbol}".upper()+'.EQ-NSE'

    ddf = ddf.sort_values(by=['Datetime'])
    ddf['Time'] = pd.to_datetime(ddf['Datetime']).dt.time
    ddf.reset_index(drop=True,inplace=True)
    ddf = ddf.rename(columns={'Adj_Open':'EQ_Open','Adj_High':'EQ_High','Adj_Low':'EQ_Low','Adj_Close':'EQ_Close','Adj_Volume':'EQ_Volume'})
    ddf = ddf[['Ticker','Date','Time','EQ_Open','EQ_High','EQ_Low','EQ_Close','EQ_Volume']]
    ddf.to_csv(fr"C:\\users\\{admin_path}\\desktop\\Stocks_Data\\{symbol.upper()}_EQ_15min.csv",index=False)

def five_underlying(ddf,symbol):
    print("CONVERTING TO 5min")
    final_df = ddf.copy()
    final_df['Date'] = pd.to_datetime(final_df['Date'], dayfirst=True)
    final_df = final_df[(final_df['Time']>=time1) & (final_df['Time']<=time2)]
    final_df.reset_index(drop=True,inplace=True)
    final_df = final_df.sort_values(by=['Date'])
    final_df = final_df.rename(columns={'Time' : 'Timestamp',
                                        'EQ_Open' : 'Adj_Open',
                                        'EQ_High' : 'Adj_High',
                                        'EQ_Low' : 'Adj_Low',
                                        'EQ_Close' : 'Adj_Close',
                                        'EQ_Volume' : 'Adj_Volume'})
    final_df['Date'] = pd.to_datetime(final_df['Date'],dayfirst=True).dt.date
    final_df = final_df.sort_values(by=['Date', 'Timestamp'])
    final_df['Date'] = final_df['Date'].astype(str)
    final_df['Timestamp'] = final_df['Timestamp'].astype(str)
    final_df['Datetime'] = pd.to_datetime(final_df['Date'] + ' ' + final_df['Timestamp'], dayfirst=True)

    final_df = final_df.set_index("Datetime")

    ddf = final_df.groupby(['Date', pd.Grouper(freq='5min')]).agg({"Adj_Open" : "first", 
                                                          "Adj_High" : "max",
                                                          "Adj_Low" : "min",
                                                          "Adj_Close" : "last", 
                                                          'Adj_Volume' : 'sum'})
    ddf.columns = ["Adj_Open", "Adj_High", "Adj_Low", "Adj_Close", 'Adj_Volume']
    ddf = ddf.reset_index()
    ddf['Ticker'] = f"{symbol}".upper()+'.EQ-NSE'

    ddf = ddf.sort_values(by=['Datetime'])
    ddf['Time'] = pd.to_datetime(ddf['Datetime']).dt.time
    ddf.reset_index(drop=True,inplace=True)
    ddf = ddf.rename(columns={'Adj_Open':'EQ_Open','Adj_High':'EQ_High','Adj_Low':'EQ_Low','Adj_Close':'EQ_Close','Adj_Volume':'EQ_Volume'})
    ddf = ddf[['Ticker','Date','Time','EQ_Open','EQ_High','EQ_Low','EQ_Close','EQ_Volume']]
    ddf.to_csv(fr"C:\\users\\{admin_path}\\desktop\\Stocks_Data\\{symbol.upper()}_EQ_5min.csv",index=False)

def one_underlying(ddf,symbol):
    final_df = ddf.copy()
    final_df['Date'] = pd.to_datetime(final_df['Date'], dayfirst=True)
    final_df = final_df[(final_df['Time']>=time1) & (final_df['Time']<=time2)]
    final_df = final_df.sort_values(by=['Date','Time'])
    final_df.reset_index(drop=True,inplace=True)
    final_df = final_df[['Ticker','Date','Time','EQ_Open','EQ_High','EQ_Low','EQ_Close','EQ_Volume']]
    final_df.to_csv(fr"C:\\users\\{admin_path}\\desktop\\Stocks_Data\\{symbol.upper()}_EQ_1min.csv",index=False)

################################# CONVERTING 1-min TO DIFFERENT TIMEFRAMES - OPTIONS DATA ##################################
def EOD(ddf,symbol,schema):
    print("CONVERTING TO EOD")
    ddf['Date'] = pd.to_datetime(ddf['Date'], dayfirst=True)
    ddf = ddf.sort_values(by=['Date'])
    
    j='-' + schema
    
    final_df = ddf.copy()
    final_df['Final_strike'] = final_df['Ticker'].str.replace(j, '')
    final_df['Final_strike'] = final_df['Final_strike'].str.replace(f'{symbol.upper()}','').str.replace('CE', '').str.replace('PE', '')
    final_df['Final_strike'] = final_df['Final_strike'].astype(float)
    final_df['Option_Type'] = final_df['Ticker'].str[-2:]
    
    final_df = final_df.rename(columns={'Time' : 'Timestamp',
                                        'Open' : 'Adj_Open',
                                        'High' : 'Adj_High',
                                        'Low' : 'Adj_Low',
                                        'Close' : 'Adj_Close',
                                        'Volume' : 'Adj_Volume',
                                        'Open_Interest' : 'Adj_OI',        
                                        'Option_type' : 'Option_Type'})
    final_df['Date'] = pd.to_datetime(final_df['Date'],dayfirst=True).dt.date
    final_df = final_df.sort_values(by=['Date', 'Timestamp'])
    final_df['Date'] = final_df['Date'].astype(str)
    final_df['Timestamp'] = final_df['Timestamp'].astype(str)
    final_df['Datetime'] = pd.to_datetime(final_df['Date'] + ' ' + final_df['Timestamp'], dayfirst=True)

    final_df = final_df.set_index("Datetime")
    final_df['Adj_OI_1'] = final_df['Adj_OI']

    df_eod = final_df.groupby(['Final_strike', 'Option_Type', pd.Grouper(freq='B')]).agg({"Adj_Open" : "first", 
                                                          "Adj_High" : "max",
                                                          "Adj_Low" : "min",
                                                          "Adj_Close" : "last", 
                                                          'Adj_Volume' : 'sum',
                                                          'Adj_OI' : 'first',
                                                          'Adj_OI_1' : 'last'})
    df_eod.columns = ["Adj_Open", "Adj_High", "Adj_Low", "Adj_Close", 'Adj_Volume', 'First_OI', 'Last_OI']
    df_eod = df_eod.reset_index()
    df_eod['rem'] = df_eod['Final_strike']%df_eod['Final_strike'].astype(int)
    df_eod.loc[df_eod['rem'] == 0, 'Ticker'] = symbol.upper() + j +  df_eod['Final_strike'].astype(int).astype(str) + df_eod['Option_Type']
    df_eod.loc[df_eod['rem'] != 0, 'Ticker'] = symbol.upper() + j + df_eod['Final_strike'].round(2).astype(str) + df_eod['Option_Type'] 

    df_eod = df_eod.sort_values(by=['Datetime', 'Final_strike'])
    df_eod = df_eod.rename(columns={'Datetime' : 'Date'})
    df_eod['Date'] = pd.to_datetime(df_eod['Date'],dayfirst=True)
    ## CHECKING IF NULL VALUES
    df_eod['New_OI'] = df_eod['Last_OI']
    df_eod = df_eod.rename(columns={'Adj_Open':'Open','Adj_High':'High','Adj_Low':'Low','Adj_Close':'Close','Adj_Volume':'Volume','New_OI':'Open_Interest'})
    df_eod = df_eod.drop(['First_OI','Last_OI','Option_Type','Final_strike','rem'],axis=1)
    df_eod = df_eod[['Ticker','Date','Open','High','Low','Close','Volume','Open_Interest']]
    df_eod.reset_index(drop=True,inplace=True)
    df_eod.to_csv(fr"C:\\users\\{admin_path}\\desktop\\Stocks_Data\\{symbol.upper()}_Opt_EOD.csv", mode='a', header = not os.path.exists(fr"C:\\users\\{admin_path}\\desktop\\Stocks_Data\\{symbol.upper()}_Opt_EOD.csv"), index=False)
    
def fifteen(ddf,symbol,schema):
    print("CONVERTING TO 15Min")
    ddf['Date'] = pd.to_datetime(ddf['Date'], dayfirst=True)
    ddf = ddf.sort_values(by=['Date'])
    
    j='-' + schema
    
    final_df = ddf.copy()
    final_df['Final_strike'] = final_df['Ticker'].str.replace(j, '')
    final_df['Final_strike'] = final_df['Final_strike'].str.replace(f'{symbol.upper()}','').str.replace('CE', '').str.replace('PE', '')
    final_df['Final_strike'] = final_df['Final_strike'].astype(float)
    final_df['Option_Type'] = final_df['Ticker'].str[-2:]
    
    final_df = final_df.rename(columns={'Time' : 'Timestamp',
                                        'Open' : 'Adj_Open',
                                        'High' : 'Adj_High',
                                        'Low' : 'Adj_Low',
                                        'Close' : 'Adj_Close',
                                        'Volume' : 'Adj_Volume',
                                        'Open_Interest' : 'Adj_OI',        
                                        'Option_type' : 'Option_Type'})
    final_df['Date'] = pd.to_datetime(final_df['Date'],dayfirst=True).dt.date
    final_df = final_df.sort_values(by=['Date', 'Timestamp'])
    final_df['Date'] = final_df['Date'].astype(str)
    final_df['Timestamp'] = final_df['Timestamp'].astype(str)
    final_df['Datetime'] = pd.to_datetime(final_df['Date'] + ' ' + final_df['Timestamp'], dayfirst=True)

    final_df = final_df.set_index("Datetime")
    final_df['Adj_OI_1'] = final_df['Adj_OI']

    df_eod = final_df.groupby(['Final_strike', 'Option_Type', pd.Grouper(freq='15min')]).agg({"Adj_Open" : "first", 
                                                          "Adj_High" : "max",
                                                          "Adj_Low" : "min",
                                                          "Adj_Close" : "last", 
                                                          'Adj_Volume' : 'sum',
                                                          'Adj_OI' : 'first',
                                                          'Adj_OI_1' : 'last'})
    df_eod.columns = ["Adj_Open", "Adj_High", "Adj_Low", "Adj_Close", 'Adj_Volume', 'First_OI', 'Last_OI']
    df_eod = df_eod.reset_index()
    df_eod['rem'] = df_eod['Final_strike']%df_eod['Final_strike'].astype(int)
    df_eod.loc[df_eod['rem'] == 0, 'Ticker'] = symbol.upper() + j +  df_eod['Final_strike'].astype(int).astype(str) + df_eod['Option_Type']
    df_eod.loc[df_eod['rem'] != 0, 'Ticker'] = symbol.upper() + j + df_eod['Final_strike'].round(2).astype(str) + df_eod['Option_Type'] 

    df_eod = df_eod.sort_values(by=['Datetime', 'Final_strike'])
    df_eod = df_eod.rename(columns={'Datetime' : 'Date'})
    df_eod['Date'] = pd.to_datetime(df_eod['Date'],dayfirst=True)
    ## CHECKING IF NULL VALUES
    df_eod['New_OI'] = df_eod['Last_OI']
    df_eod = df_eod.rename(columns={'Adj_Open':'Open','Adj_High':'High','Adj_Low':'Low','Adj_Close':'Close','Adj_Volume':'Volume','New_OI':'Open_Interest'})
    df_eod = df_eod.drop(['First_OI','Last_OI','Option_Type','Final_strike','rem'],axis=1)
    df_eod = df_eod[['Ticker','Date','Open','High','Low','Close','Volume','Open_Interest']]
    df_eod.reset_index(drop=True,inplace=True)
    df_eod.to_csv(fr"C:\\users\\{admin_path}\\desktop\\Stocks_Data\\{symbol.upper()}_Opt_15min.csv", mode='a', header = not os.path.exists(fr"C:\\users\\{admin_path}\\desktop\\Stocks_Data\\{symbol.upper()}_Opt_15min.csv"), index=False)

def five(ddf,symbol,schema):
    print("CONVERTING TO 5Min")
    ddf['Date'] = pd.to_datetime(ddf['Date'], dayfirst=True)
    ddf = ddf.sort_values(by=['Date'])
    
    j='-' + schema
    
    final_df = ddf.copy()
    final_df['Final_strike'] = final_df['Ticker'].str.replace(j, '')
    final_df['Final_strike'] = final_df['Final_strike'].str.replace(f'{symbol.upper()}','').str.replace('CE', '').str.replace('PE', '')
    final_df['Final_strike'] = final_df['Final_strike'].astype(float)
    final_df['Option_Type'] = final_df['Ticker'].str[-2:]
    
    final_df = final_df.rename(columns={'Time' : 'Timestamp',
                                        'Open' : 'Adj_Open',
                                        'High' : 'Adj_High',
                                        'Low' : 'Adj_Low',
                                        'Close' : 'Adj_Close',
                                        'Volume' : 'Adj_Volume',
                                        'Open_Interest' : 'Adj_OI',        
                                        'Option_type' : 'Option_Type'})
    final_df['Date'] = pd.to_datetime(final_df['Date'],dayfirst=True).dt.date
    final_df = final_df.sort_values(by=['Date', 'Timestamp'])
    final_df['Date'] = final_df['Date'].astype(str)
    final_df['Timestamp'] = final_df['Timestamp'].astype(str)
    final_df['Datetime'] = pd.to_datetime(final_df['Date'] + ' ' + final_df['Timestamp'], dayfirst=True)

    final_df = final_df.set_index("Datetime")
    final_df['Adj_OI_1'] = final_df['Adj_OI']

    df_eod = final_df.groupby(['Final_strike', 'Option_Type', pd.Grouper(freq='5min')]).agg({"Adj_Open" : "first", 
                                                          "Adj_High" : "max",
                                                          "Adj_Low" : "min",
                                                          "Adj_Close" : "last", 
                                                          'Adj_Volume' : 'sum',
                                                          'Adj_OI' : 'first',
                                                          'Adj_OI_1' : 'last'})
    df_eod.columns = ["Adj_Open", "Adj_High", "Adj_Low", "Adj_Close", 'Adj_Volume', 'First_OI', 'Last_OI']
    df_eod = df_eod.reset_index()
    df_eod['rem'] = df_eod['Final_strike']%df_eod['Final_strike'].astype(int)
    df_eod.loc[df_eod['rem'] == 0, 'Ticker'] = symbol.upper() + j +  df_eod['Final_strike'].astype(int).astype(str) + df_eod['Option_Type']
    df_eod.loc[df_eod['rem'] != 0, 'Ticker'] = symbol.upper() + j + df_eod['Final_strike'].round(2).astype(str) + df_eod['Option_Type'] 

    df_eod = df_eod.sort_values(by=['Datetime', 'Final_strike'])
    df_eod = df_eod.rename(columns={'Datetime' : 'Date'})
    df_eod['Date'] = pd.to_datetime(df_eod['Date'],dayfirst=True)
    ## CHECKING IF NULL VALUES
    df_eod['New_OI'] = df_eod['Last_OI']
    df_eod = df_eod.rename(columns={'Adj_Open':'Open','Adj_High':'High','Adj_Low':'Low','Adj_Close':'Close','Adj_Volume':'Volume','New_OI':'Open_Interest'})
    df_eod = df_eod.drop(['First_OI','Last_OI','Option_Type','Final_strike','rem'],axis=1)
    df_eod = df_eod[['Ticker','Date','Open','High','Low','Close','Volume','Open_Interest']]
    df_eod.reset_index(drop=True,inplace=True)
    df_eod.to_csv(fr"C:\\users\\{admin_path}\\desktop\\Stocks_Data\\{symbol.upper()}_Opt_5min.csv", mode='a', header = not os.path.exists(fr"C:\\users\\{admin_path}\\desktop\\Stocks_Data\\{symbol.upper()}_Opt_5min.csv"), index=False)

def one(ddf,symbol,schema):
    
    ddf['Date'] = pd.to_datetime(ddf['Date'], dayfirst=True)
    ddf = ddf.sort_values(by=['Date'])
    ddf = ddf[(ddf['Time']>=time1) & ((ddf['Time']<=time2))]
    ddf = ddf.sort_values(by=['Date','Time'])
    ddf.reset_index(drop=True,inplace=True)
    ddf = ddf[['Ticker','Date','Time','Open','High','Low','Close','Volume','Open_Interest']]
    ddf.to_csv(fr"C:\\users\\{admin_path}\\desktop\\Stocks_Data\\{symbol.upper()}_Opt_1min.csv", mode='a', header = not os.path.exists(fr"C:\\users\\{admin_path}\\desktop\\Stocks_Data\\{symbol.upper()}_Opt_1min.csv"), index=False)

####################################### UNDERLYING DATA FUNCTION ##########################################
def underlying_data(symbol,date1,date2,conversion):
    print("\nFetching Stock Equity Data")
    ddf = pd.DataFrame()
    engine = pg.connect("dbname='StockEQ' user='postgres' host='swandatabase.cfehmk2wtejq.ap-south-1.rds.amazonaws.com' port='5432' password='swancap123'")
    st_t = time.time()
    ddf = pd.read_sql(f'select * from "AdjustedData"."{symbol.upper()}" where "Date" between \'{date1}\' and \'{date2}\'', con=engine)
    ddf['Ticker'] = f'{symbol.upper()}' + '.EQ-NSE'
    ddf = ddf.sort_values(by=["Date",'Time'])
    
    if conversion == 'E':
        EOD_underlying(ddf,symbol)
    elif conversion == '15':
        fifteen_underlying(ddf,symbol)
    elif conversion == '5':
        five_underlying(ddf,symbol)
    elif conversion == '1':
        one_underlying(ddf,symbol)
        
    et_t = time.time()
    print("Time taken to process",et_t-st_t)
    engine.close()

####################################### OPTION DATA FUNCTION ##########################################        
def option_data(symbol,date1,date2,conversion):
    print("\nFetching Stock Option Data")
    schema = input("Enter schema (I/II/III) - ")
    date1 = datetime.strptime(date1, "%Y-%m-%d").date()
    date2 = datetime.strptime(date2, "%Y-%m-%d").date()
    year1 = date1.year
    year2 = date2.year
    
    st_t = time.time()
    for i in tqdm(range(int(year1),int(year2)+1)):
        ddate1 = '-01-01'
        ddate2 = '-12-31'
        year_start = str(str(i)+ddate1)
        year_end = str(str(i)+ddate2)
        year_start = datetime.strptime(year_start, "%Y-%m-%d").date()
        year_end = datetime.strptime(year_end, "%Y-%m-%d").date()
        if date1 > year_start :
            year_start = date1
        else:
            year_start = year_start
        if year_end > date2 :
            year_end = date2
        else:
            year_end = year_end
        
        ddf = pd.DataFrame()
        engine = pg.connect("dbname='StockOptions' user='postgres' host='swandatabase.cfehmk2wtejq.ap-south-1.rds.amazonaws.com' port='5432' password='swancap123'")
        print("Generating data from ", year_start , " to " , year_end)
        ddf = pd.read_sql(f'select * from "Monthly{schema}"."{symbol.upper()}-{schema}" where "Date" between \'{year_start}\' and \'{year_end}\'', con=engine)
        ddf = ddf.sort_values(by=["Date",'Time'])
        ddf.reset_index(drop=True,inplace=True)
#         ddf.to_csv(rf"C:\\users\\admin\\desktop\\Stocks_Data\\{symbol.upper()}_"+str(i)+".csv",index=False)
        
        if conversion == 'E':
            EOD(ddf,symbol,schema)
        elif conversion == '15':
            fifteen(ddf,symbol,schema)
        elif conversion == '5':
            five(ddf,symbol,schema)
        elif conversion == '1':
            one(ddf,symbol,schema)

    et_t = time.time()
    print("Time taken to process",et_t-st_t)
    engine.close()
    
###################################### MAIN CODE STARTS FROM HERE ############################################
admin_path = 'admin'
data = input("Enter O for Options data, U for Underlying data, B for Both the data, E for Exiting ")

time1 = datetime.strptime('09:15:00','%H:%M:%S').time()
time2 = datetime.strptime('15:30:00','%H:%M:%S').time()

## CREATING A DIRECTORY OF THE REQUIRED INDEX AND SCHEMA
if not os.path.exists(fr"C:\\users\\{admin_path}\\desktop\\Stocks_Data\\"):
    os.makedirs(fr"C:\users\\{admin_path}\desktop\\Stocks_Data\\")

#################################### TAKING INPUTS FOR SYMBOLS, DATE RANGE AND TIMEFRAME #############################

if data == 'o' or data == 'O' or data == 'u' or data == 'U' or data == 'b' or data == 'B':
    symbol = input('Enter the Symbol ')
    date1 = input("Enter Start date as YYYY-MM-DD ")
    date2 = input("Enter End date as YYYY-MM-DD ")
    conversion = input("Enter 1 for 1 minute, 5 for 5 minutes, 15 for 15 minutes, E for EOD ")
    
elif data == 'e' or data == 'E':
    print("Exit!")

else:
    print("Wrong option")

st=time.time()

if data == 'o' or data == 'O':
    option_data(symbol,date1,date2,conversion)

elif data == 'U' or data == 'u':
    underlying_data(symbol,date1,date2,conversion)

elif data == 'b' or data == 'B':
    option_data(symbol,date1,date2,conversion)
    underlying_data(symbol,date1,date2,conversion)
    
et = time.time()
print("\nTotal time taken ",et-st)

Enter O for Options data, U for Underlying data, B for Both the data, E for Exiting o
Enter the Symbol reliance
Enter Start date as YYYY-MM-DD 2023-01-01
Enter End date as YYYY-MM-DD 2023-05-31
Enter 1 for 1 minute, 5 for 5 minutes, 15 for 15 minutes, E for EOD 15

Fetching Stock Option Data
Enter schema (I/II/III) - I


  0%|          | 0/1 [00:00<?, ?it/s]

Generating data from  2023-01-01  to  2023-05-31
CONVERTING TO 15Min
Time taken to process 30.479538202285767

Total time taken  32.189889907836914
